In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression #For regression model
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [3]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [4]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [5]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [6]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [7]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [8]:
print('training set prep')
train_df=seperate_monthly(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)

training set prep
test set prep


In [9]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [10]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [11]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [12]:
Regresion_model= model = LinearRegression() #Define model object

In [13]:
X_train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
93336,0.248410,0.409091,0.166667,0.096605,0.162140,0.207050,0.214136,0.176625,0.150769,0.222164,...,-0.214340,-0.120096,-0.299224,-0.822398,0.252820,0.093125,0.226167,0.134108,0.083333,1.0000
2127,0.005661,0.045455,0.833333,0.445458,0.761641,0.878157,0.787361,0.814048,0.738741,0.833116,...,-0.125333,0.273104,-0.220863,-0.475396,-0.242711,0.294510,0.142411,0.355403,0.500000,1.0000
94189,0.250681,0.409091,0.200000,0.517247,0.372506,0.393283,0.368973,0.385893,0.356084,0.211473,...,0.306105,-0.176765,-0.375099,-0.307881,0.113322,0.689969,0.137989,-0.079066,0.416667,1.0000
135652,0.361033,0.500000,0.233333,0.189454,0.276885,0.330003,0.294400,0.363159,0.308252,0.291786,...,0.625428,0.398649,-0.531722,0.081165,0.049479,0.030528,0.204578,0.186781,0.833333,0.9375
47639,0.126790,0.272727,0.666667,0.075662,0.082594,0.222037,0.025157,0.145730,0.142098,0.134029,...,0.099372,0.458915,-0.135442,-0.631033,-0.059268,0.053232,0.353102,-0.155876,0.083333,0.9375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95238,0.253473,0.409091,0.266667,0.311870,0.168514,0.214543,0.263552,0.269309,0.334545,0.282920,...,0.155382,0.150937,-0.559666,-0.340026,0.091934,0.033094,0.235921,0.604106,0.250000,0.9375
26953,0.071734,0.227273,0.466667,0.578025,0.603381,0.590896,0.647499,0.544448,0.534266,0.590613,...,-0.102891,-0.153662,0.404336,0.218232,-0.024686,0.376732,0.484458,0.247011,0.416667,1.0000
62960,0.167566,0.318182,0.566667,0.138895,0.103381,0.204552,0.024858,0.157971,0.156643,0.100913,...,-0.138361,-0.397777,-0.290473,0.222191,-0.878804,0.385711,0.167122,0.387435,1.000000,0.8750
161166,0.428938,0.545455,0.433333,0.489208,0.695399,0.709686,0.733753,0.781988,0.636643,0.655802,...,-0.403671,0.279207,0.042064,-0.023096,-0.525750,0.834787,-0.299389,-0.284345,0.666667,0.9375


In [14]:
model.fit(X_train_df, y_train_df)

LinearRegression()

In [15]:
predictions=model.predict(X_train_df)

In [16]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [17]:
i=375734
final['index']=0
while i<728337:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,7.401152,375734
1,30.704950,375735
2,14.183668,375736
3,8.694157,375737
4,1.540391,375738
...,...,...
352599,16.375001,728333
352600,26.000465,728334
352601,6.376756,728335
352602,20.628753,728336


In [18]:
final.to_csv('/kaggle/working/regression_wetherprediction.csv',index=False)